In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import os
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.cell import Cell
from lib.spec import Spec
from lib.model import NModel




(train_validate_images, train_validate_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_images ,validate_images = np.split(train_validate_images, [40000])
train_labels ,validate_labels = np.split(train_validate_labels, [40000])

# Normalize pixel values to be between 0 and 1
train_images, validate_images, test_images = train_images / 255.0, validate_images / 255.0,test_images / 255.0

In [2]:

cf = Spec.get_spec_from_json('{"edge_0": 0, "edge_1": 0, "edge_10": 1, "edge_11": 0, "edge_12": 0, "edge_13": 1, "edge_14": 0, "edge_15": 0, "edge_16": 1, "edge_17": 1, "edge_18": 1, "edge_19": 1, "edge_2": 1, "edge_20": 0, "edge_3": 1, "edge_4": 1, "edge_5": 0, "edge_6": 0, "edge_7": 0, "edge_8": 1, "edge_9": 0, "op_node_0": "conv3x3-bn-relu", "op_node_1": "conv1x1-bn-relu", "op_node_2": "conv1x1-bn-relu", "op_node_3": "conv3x3-bn-relu", "op_node_4": "conv3x3-bn-relu"}')

s = Spec(cf)

print(s.matrix)
print(s.ops)

s.vi

[[0 1 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'output']


In [4]:
i = tf.keras.Input(shape=(32, 32, 3))
nModel = NModel(spec=s, name="test")
model = nModel.build()
model.summary()

___________________
concatenate_1 (Concatenate)     (None, 32, 32, 128)  0           re_lu_5[0][0]                    
                                                                 re_lu_7[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 32, 32, 64)   8192        concatenate_1[0][0]              
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 32, 32, 64)   8192        concatenate_1[0][0]              
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 32, 32, 64)   256         conv2d_9[0][0]                   
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 32, 32, 64)   256         conv2d_11[0][0]         

In [6]:
data = nModel.train_and_evaluate()
data

Train on 40000 samples, validate on 10000 samples
Epoch 1/12
 1024/40000 [..............................] - ETA: 22:49 - loss: 2.3323 - accuracy: 0.1224

KeyboardInterrupt: 

In [15]:
history.history

{'loss': [1.8895489448547362,
  1.4495863836288452,
  1.402880509185791,
  1.402955818748474],
 'accuracy': [0.3231, 0.46195, 0.4816, 0.480025],
 'val_loss': [107566.1637,
  192.51275043945313,
  17.174112457275392,
  3.8490982215881346],
 'val_accuracy': [0.1027, 0.1282, 0.1211, 0.1787]}

In [16]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 4s 403us/sample - loss: 3.9102 - accuracy: 0.1745


[3.9102051452636717, 0.1745]

In [17]:
print(model.count_params())
np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_variables])

3742346


3728906

In [18]:
model.history.params

{'batch_size': 256,
 'epochs': 4,
 'steps': 157,
 'samples': 40000,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'accuracy', 'val_loss', 'val_accuracy']}

In [22]:
model.history.history

{'loss': [1.4638422157287598,
  1.1612712270736694,
  1.1123817365646362,
  1.1109454036712647],
 'accuracy': [0.4653, 0.57965, 0.59985, 0.6013],
 'val_loss': [3.754738680267334,
  7.283068478393555,
  3.8965704246521,
  2.236551708984375],
 'val_accuracy': [0.1128, 0.1562, 0.27, 0.3833]}

In [20]:
model.history.history["accuracy"][-1]

0.480025

In [21]:
data = nModel.train_and_evaluate()
data

Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 40s 992us/sample - loss: 1.4638 - accuracy: 0.4653 - val_loss: 3.7547 - val_accuracy: 0.1128
Epoch 2/4
40000/40000 [==============================] - 36s 895us/sample - loss: 1.1613 - accuracy: 0.5796 - val_loss: 7.2831 - val_accuracy: 0.1562
Epoch 3/4
40000/40000 [==============================] - 36s 899us/sample - loss: 1.1124 - accuracy: 0.5998 - val_loss: 3.8966 - val_accuracy: 0.2700
Epoch 4/4
10000/10000 [==============================] - 4s 391us/sample - loss: 2.2466 - accuracy: 0.3829


{'trainable_parameters': 3728906,
 'params': {'batch_size': 256,
  'epochs': 4,
  'steps': 157,
  'samples': 40000,
  'verbose': 0,
  'do_validation': True,
  'metrics': ['loss', 'accuracy', 'val_loss', 'val_accuracy']},
 'history': {'loss': [1.4638422157287598,
   1.1612712270736694,
   1.1123817365646362,
   1.1109454036712647],
  'accuracy': [0.4653, 0.57965, 0.59985, 0.6013],
  'val_loss': [3.754738680267334,
   7.283068478393555,
   3.8965704246521,
   2.236551708984375],
  'val_accuracy': [0.1128, 0.1562, 0.27, 0.3833]},
 'training_time': 147.2608449459076,
 'train_accuracy': 0.6013,
 'validation_accuracy': 0.3833,
 'test_accuracy': 0.3829}